In [9]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Model

In [10]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

assert x_train.shape == (50000, 32, 32, 3)
assert x_test.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 1)
assert y_test.shape == (10000, 1)


def preprocess(image):
    image = tf.image.resize(image, (224, 224))
    image = tf.cast(image, tf.float32) / 255.0
    #image = image[None, ...]
    return image

batch_size = 32

train_dataset = tf.data.Dataset.from_tensor_slices(x_train)
train_dataset = train_dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.batch(batch_size).shuffle(buffer_size=10000).prefetch(buffer_size=tf.data.AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices(x_test)
test_dataset = test_dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)

print(train_dataset)


<_PrefetchDataset element_spec=TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None)>


In [11]:
class Autoencoder(Model):
    def __init__(self, shape):
        super(Autoencoder, self).__init__()
        self.shape = shape
        self.encoder = tf.keras.Sequential([
            layers.Input(shape=(224, 224, 3)),
            layers.Conv2D(112, (3, 3), activation='leaky_relu', padding='same', strides=2),
            layers.MaxPool2D((2,2), padding='same'),
            layers.Conv2D(56, (3, 3), activation='leaky_relu', padding='same', strides=2),
            layers.Conv2D(28, (3, 3), activation='leaky_relu', padding='same', strides=2),
            layers.Conv2D(28, (3, 3), activation='leaky_relu', padding='same', strides=2)
        ])
        self.decoder = tf.keras.Sequential([
            layers.Conv2DTranspose(28, (3, 3), activation='leaky_relu', padding='same', strides=2),
            layers.Conv2DTranspose(28, (3, 3), activation='leaky_relu', padding='same', strides=2),
            layers.Conv2DTranspose(56, (3, 3), activation='leaky_relu', padding='same', strides=2),
            layers.UpSampling2D(size=(2,2), interpolation='nearest'),
            layers.Conv2DTranspose(112, (3, 3), activation='leaky_relu', padding='same', strides=2),
            layers.Conv2D(3, kernel_size=(3,3), activation='sigmoid', padding='same')
        ])
    
    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

shape = (224, 224, 3)
autoencoder = Autoencoder(shape)

In [14]:
autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())

autoencoder.fit(train_dataset,
                epochs=10,
                shuffle=True,
                validation_data=test_dataset)

Epoch 1/10


ValueError: None values not supported.